In [8]:
import glob

from src.create_annotations import *

# Label ids of the dataset
# category_ids = {
#     "outlier": 0,
#     "window": 1,
#     "wall": 2,
#     "balcony": 3,
#     "door": 4,
#     "roof": 5,
#     "sky": 6,
#     "shop": 7,
#     "chimney": 8
# }

category_ids = {
    "road": 0,
    "person": 1,
    "car": 2,
    "grass": 3,
    "building": 4,
    "vegetation": 5,
    "pool": 6,
    "sky": 7,
    "road line": 8
}

# Define which colors match which categories in the images
# category_colors = {
#     "(0, 0, 0)": 0, # Outlier
#     "(255, 0, 0)": 1, # Window
#     "(255, 255, 0)": 2, # Wall
#     "(128, 0, 255)": 3, # Balcony
#     "(255, 128, 0)": 4, # Door
#     "(0, 0, 255)": 5, # Roof
#     "(128, 255, 255)": 6, # Sky
#     "(0, 255, 0)": 7, # Shop
#     "(128, 128, 128)": 8 # Chimney
# }

category_colors = {
#     "(128, 64, 128)": 0, # Road
    "(128, 64, 128)": 0, # Road
#     "(232, 35, 244)": 0, # Road
    "(244, 35, 232)": 0, # Road
    "(0, 0, 142)": 2, # Car
#     "(100, 170, 145)": 2, # Grass
    "(145, 170, 100)": 3, # Grass
#     "(142, 0, 0)": 3, # Car
    
    "(220, 20, 60)": 6, 
    "(0, 0, 0)": 1, # Car
#     "(180, 130, 70)": 7, # Sky
    "(70, 130, 180)": 7, # Sky
#     "(50, 234, 157)": 8 # Road Line
    "(157, 234, 50)": 8 # Road Line
}

# Define the ids that are a multiplolygon. In our case: wall, roof and sky
multipolygon_ids =[1]

# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []
    
    for mask_image in glob.glob(maskpath + "*.png"):
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image).split(".")[0] + ".png"

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info 
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, w, h)
        for color, sub_mask in sub_masks.items():
            category_id = category_colors[color]

            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)

            # Check if we have classes that are a multipolygon
            if category_id in multipolygon_ids:
                # Combine the polygons to calculate the bounding box and area
                multi_poly = MultiPolygon(polygons)
                print("annotation_id",annotation_id)
                print("category_id",category_id)
                print("segmentations",segmentations)
                print("multi_poly",multi_poly)
                print("image_id",image_id)
                try:
                    annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)
                except:
                    pass
                annotations.append(annotation)
                annotation_id += 1
            else:
                for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id

if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    
    for keyword in ["seg_copy","seg", "val","simtest"]:
#         mask_path = "dataset/lunar/mock_sim/{}/".format(keyword)
        mask_path = "./dataset/{}/".format(keyword)
#         mask_path = "./dataset/lunar/mock_sim/{}/".format(keyword)
        
        # Create category section
        coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

        with open("output/{}.json".format(keyword),"w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))


Created 0 annotations for images in folder: ./dataset/seg_copy/
Created 0 annotations for images in folder: ./dataset/seg/
Created 0 annotations for images in folder: ./dataset/val/
Created 793 annotations for images in folder: ./dataset/simtest/
